In [ ]:
from keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import random
!python --version

Python 3.7.13


跟google雲端連接

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#讀檔--給一個資料夾會讀所有底下的檔，包含子資料夾
#.CRI是超音波圖
#.png是標記(從原始資料用matlab轉換來的)
import cv2
import scipy.io
#[50:500 100:700]
def load_images_from_folder(path):
  image_name = []
  image = []
  mask = []
  original_sizes = [] #因為之後會resize img，這邊可以用來復原
  i=0
  files= os.listdir(path) #得到資料夾下的所有檔名稱
  for root, dirs, files in os.walk(path):
    for file in files:
      fullpath = os.path.join(root, file)
      if file.find(".CRI") != -1:
        img = cv2.imread(fullpath, cv2.IMREAD_GRAYSCALE)
        original_sizes.append(img.shape)
        img = img[50:500, 100:700] #擷取血管的大致位置(主要是把邊框切掉)
        print(file +"  "+ str(i))
        i+=1
        image_name.append(file[:-4])
        image.append(img)
      elif file.find(".png") != -1:
        img = cv2.imread(fullpath, cv2.IMREAD_GRAYSCALE)
        img = img[50:500, 100:700] 
        mask.append(img)
  image = np.array(image)
  mask = np.array(mask)
  return image, mask, image_name, original_sizes

In [ ]:
from scipy.signal import argrelextrema
from scipy.ndimage.filters import gaussian_filter

# input: img
# return: cropImgUp(沿血管中心水平裁切後的img上半部+垂直翻轉 aka翻轉後的血管上壁)
#         cropImgDown(沿血管中心水平裁切後的img下半部 aka血管下壁)
#         cropLineUp(上半部img的上界)
#         axis(血管的軸 aka第幾個row)
#         cropLineDown(下半部img的下界)

def get_cropImg_axis(img):

    # 把img每個row所有pixel的intensity value相加存進row_sum
    row_size = img.shape[0]
    row_sum = np.zeros(row_size)
    for i in range(row_size):
        row_sum[i] = np.sum(img[i,:])
    row_sum = gaussian_filter(row_sum, sigma=7)

    # 從row_sum中找出所有的local maximum
    local_max = argrelextrema(row_sum, np.greater)
    max_value =  row_sum[local_max]   
    max_value = np.sort(max_value)

    # 如果local maximum的數量>=3，就取出值前三大的local maximum，假設為血管壁位置
    if len(max_value)>=3:
        max1 = max_value[-1]
        max2 = max_value[-2]
        max3 = max_value[-3]
        for i in range(row_size):
            if row_sum[i] == max1:
                wall1 = i
            elif row_sum[i] == max2:
                wall2 = i
            elif row_sum[i] == max3:
                wall3 = i
        # 上下血管壁位置取中間即為血管軸
        # 算出三組可能為血管軸的位置，找出intensity value最小者
        # 並記錄下此狀況下，血管壁的位置
        axis1 = (wall1 + wall2)//2
        axis2 = (wall2 + wall3)//2
        axis3 = (wall1 + wall3)//2
        axis = axis1
        wallUp = wall1
        wallDown = wall2
        if row_sum[axis]>row_sum[axis2]:
            axis = axis2
            wallUp = wall2
            wallDown = wall3
        if row_sum[axis]>row_sum[axis3]:
            axis = axis3
            wallUp = wall1
            wallDown = wall3

    #如果local maximum的數量>=，兩血管壁位置取中間即為血管軸
    elif len(max_value)>=2:
        max1 = max_value[-1]
        max2 = max_value[-2]
        for i in range(row_size):
            if row_sum[i] == max1:
                wall1 = i
            elif row_sum[i] == max2:
                wall2 = i
        axis = (wall1 + wall2)//2
        wallUp = wall1
        wallDown = wall2

    else:
        print('error')
        return

    if wallUp>wallDown:
        temp = wallDown
        wallDown = wallUp
        wallUp = temp
    axis = int(axis)#
    # 透過血管壁與軸的位置做裁切，裁切邊界必須在img內
    cropLineUp = wallUp-(axis-wallUp)   
    if cropLineUp<0:
        cropLineUp = 0
    cropLineDown = wallDown+(wallDown-axis) 
    if cropLineDown>img.shape[0]:
        cropLineDown = img.shape[0]

    # 回傳裁切後圖片以及裁切軸

    cropImgDown = img[axis:cropLineDown,:]
    cropImgUp = img[cropLineUp:axis,:]
    cropImgUp = cv2.flip(cropImgUp, 0)

    return cropImgUp, cropImgDown, cropLineUp, axis, cropLineDown

In [ ]:
#這邊做shuffle，原本是拿來用cross validation，但後來沒用到
#shuff_const是random的seed 要記得跟train那邊的code一樣
def shuff(imgs):
  #input (78,512,512)
  shuff_const = 20010319
  indices = np.arange(imgs.shape[0])
  np.random.seed(shuff_const)
  np.random.shuffle(indices)
  return indices

In [ ]:
#sobel和prewitt filter 
def imgsToSobel(img):
  sobely = cv2.Sobel(img,cv2.CV_8U,0,1,ksize=3)
  return np.array(sobely)
#prewitt 似乎有很多不同的，這邊是弄得跟論文一樣
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8541435/
def imgsToPrewitt(img):
  #kernelx = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=int)
  kernelx = np.array([[-1.,-1.,-1.],[0.,0.,0.],[1.,1.,1.]], dtype=np.float64)
  kernely = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float64)
  x = cv2.filter2D(img.astype(np.float64), -1, kernelx,borderType = cv2.BORDER_CONSTANT)
  y = cv2.filter2D(img.astype(np.float64), -1, kernely,borderType= cv2.BORDER_CONSTANT)
  x = np.clip(x, 0, 255).astype(np.uint8)
  y = np.clip(y, 0, 255).astype(np.uint8)

  Prewitt = y-x
  return 255-Prewitt


In [ ]:
#這邊做兩件事，先經過兩個filter，再resize 成model的input size
def produce_preprocessed_and_resized_img(imgs):
  mask = []
  img = []
  pimg = []
  Resize = (512,128)
  for image in imgs:
    sobel = imgsToSobel(image)
    prewitt = imgsToPrewitt(image)
    image = cv2.resize(image, Resize, interpolation = cv2.INTER_AREA)
    sobel = cv2.resize(sobel, Resize, interpolation = cv2.INTER_AREA)
    prewitt = cv2.resize(prewitt, Resize, interpolation = cv2.INTER_AREA)
    img.append([image, sobel, prewitt])
  return np.array(img)


In [ ]:
#把validaion分出來
def seg_training_data(imgs, masks, index = 0):
  partition = int(len(imgs)*0.2)
  t_imgs = np.append(imgs[0 : index * partition], imgs[(index+1) * partition :], axis =0)
  t_masks = np.append(masks[0 : index * partition], masks[(index+1) * partition :], axis =0)

  V_imgs = imgs[index * partition : (index+1) * partition]
  V_masks = masks[index * partition : (index+1) * partition]

  return t_imgs, t_masks, V_imgs, V_masks

In [ ]:
#把上下的血管壁切出來
def get_crop(imgs, truths):
  pdown = []
  pup = []
  topimgs = []
  botimgs = []
  cropLineUp = []
  cropLineDown = []
  bot_cropped_size = []
  top_cropped_size = []
  axises = []
  i=0
  for img, truth in zip(imgs, truths):
    Resize = (512,128)
    #return cropImgUp, cropImgDown, cropLineUp, axis, cropLineDown
    top, bot, LineUp, axis, LineDown = get_cropImg_axis(img)
    top_cropped_size.append(top.shape)
    bot_cropped_size.append(bot.shape)
    

    top = cv2.resize(top, Resize, interpolation = cv2.INTER_AREA)
    bot = cv2.resize(bot, Resize, interpolation = cv2.INTER_AREA)
    
    axises.append(axis)
    topimgs.append(top)
    botimgs.append(bot)
    cropLineUp.append(LineUp)
    cropLineDown.append(LineDown)
  pup = np.array(pup)  
  pdown = np.array(pdown)
  truths = np.array(truths)
  return np.array(topimgs), np.array(botimgs), top_cropped_size, bot_cropped_size, cropLineUp, axises, cropLineDown, truths

In [ ]:
#np.random.seed(20010319)
#np.random.shuffle(img)

In [ ]:
dir_train = "drive/MyDrive/大三專題/100-IMT-ImagesCY/train"#training
imgs, masks, imgs_names, original_sizes = load_images_from_folder(dir_train)


In [ ]:
print(original_sizes)

In [ ]:
indice = shuff(imgs)
imgs = imgs[indice]
masks = masks[indice]


In [ ]:
imgs, masks, V_imgs, V_masks = seg_training_data(imgs, masks)

In [ ]:

model_path = "drive/MyDrive/大三專題/100-IMT-ImagesCY/model0811_sobel_prewitt.h5"
model = load_model(model_path)


In [ ]:
#用model標出上下血管壁內膜
def do_seg(img, sobel_img, prewitt_image):
    testimg = img
    #input1 = img.reshape(-1,128,512,1).astype("float32")/255
    input2 = sobel_img.reshape(-1,128,512,1).astype("float32")/255
    input3 = prewitt_image.reshape(-1,128,512,1).astype("float32")/255
    #seg = model.predict([input1, input2, input3])
    seg = model.predict([input2, input3])
    seg = seg.reshape(128,512)
    seg= (seg*255).astype(np.uint8)
    
    ret, seg = cv2.threshold(seg, 5, 255, cv2.THRESH_BINARY)
    #threshhold 可以設5
    #這邊要找一下有沒有更好的方法
    return seg
    

In [ ]:
topimgs, botimgs, top_cropped_size, bot_cropped_size, cropLineUp, axis, cropLineDown, V_botmasks = get_crop(V_imgs, V_masks)
topimgs = produce_preprocessed_and_resized_img(topimgs)#(79,2,512,512)
botimgs = produce_preprocessed_and_resized_img(botimgs)#(79,2,512,512)

In [ ]:
print(topimgs.shape)

In [ ]:
results = []
x = [50, 500]
y = [100, 700]
#這邊是把model predict的標記拚貼到一起
for i in range(len(topimgs[:,0])):
  top_mask = do_seg(topimgs[:,0][i], topimgs[:,1][i], topimgs[:,2][i])
  top_mask = cv2.flip(top_mask, 0)
    
  bot_mask = do_seg(botimgs[:,0][i], botimgs[:,1][i], botimgs[:,2][i])
  top_mask = cv2.resize(top_mask, (top_cropped_size[i][1], top_cropped_size[i][0]), interpolation = cv2.INTER_NEAREST)
  bot_mask = cv2.resize(bot_mask, (bot_cropped_size[i][1], bot_cropped_size[i][0]), interpolation = cv2.INTER_NEAREST)
  combined = np.append(np.array(top_mask), np.array(bot_mask), axis = 0)

  black = np.zeros(original_sizes[i])
  leftzero = np.zeros((combined.shape[0],y[0]))#補0為了把寬度弄一樣
  combined = np.append(leftzero, combined, axis = 1)
  rightzero = np.zeros((combined.shape[0],original_sizes[i][1]-combined.shape[1]))#補0為了把寬度弄一樣
  combined = np.append(combined, rightzero, axis = 1)
  black[cropLineUp[i]+x[0]:cropLineDown[i]+x[0]] = combined[:]
  #plt.imshow(black, cmap = 'gray')
  #plt.show()
  #print("---------------------------")
  results.append(black)
results = np.array(results)

In [ ]:
print(results.shape)
print(V_botmasks.shape)

In [ ]:
i=0
for img, result in zip(V_imgs, results):
  alpha =0.5
  plt.imshow(img,cmap='gray')
  plt.show()
  plt.imshow(topimgs[i][1],cmap='gray')
  plt.show()
  plt.imshow(botimgs[i][1],cmap='gray')
  plt.show()
  overlap = cv2.addWeighted(img, alpha, result[x[0]:x[1],y[0]:y[1]].astype(np.uint8), 1-alpha,0) #把影像和標記貼在一起
  plt.imshow(overlap,cmap='gray')
  plt.show()
  print("-------------" + str(i))
  i+=1

In [ ]:
#計算dice coefficient(和資料集的下血管壁比)
def dice_coef(y_true, y_pred, smooth=0):
    import keras.backend as K
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    a = 2. * intersection + smooth
    b = K.sum(y_true_f) + K.sum(y_pred_f)
    if b == 0:
      return 0
    else:
      return a/b
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
for i in range(len(results)):
  temp = results[i][x[0]:x[1],y[0]:y[1]]
  ypred = temp[axis[i]:cropLineDown[i]].astype(np.float32)/255
  ytrue = V_masks[i][axis[i]:cropLineDown[i]].astype(np.float32)/255
  plt.imshow(results[i], cmap = 'gray')
  plt.show()
  plt.imshow(ypred, cmap = 'gray')
  plt.show()
  plt.imshow(ytrue, cmap = 'gray')
  plt.show()
  print("---------------------")

In [ ]:
total = 0
for i in range(len(results)):
  temp = results[i][x[0]:x[1],y[0]:y[1]]
  ypred = temp[axis[i]:cropLineDown[i]].astype(np.float32)/255
  ytrue = V_masks[i][axis[i]:cropLineDown[i]].astype(np.float32)/255
  total = total + dice_coef(ytrue, ypred)
print(total/(len(results)-1))#有一個crop錯